In [1]:
import pandas as pd
import sqlite3
import logging

In [2]:
logging.basicConfig(filename = 'pipeline.log' , format = '%(asctime)s - %(levelname)s - %(message)s' , level = logging.DEBUG )

In [3]:
def extraction():
        """
    Extracts Walmart sales data from a CSV file into a pandas DataFrame.

    Args:
        file_path (str): Path to the CSV file containing raw Walmart sales data.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted sales data.
    
    Raises:
        FileNotFoundError: If the specified file_path does not exist.
        pd.errors.ParserError: If the CSV is malformed or unreadable.
    """
    logging.info('Starting the extraction process')
    try:
        return pd.read_csv('Walmart.csv')
        logging.info('Successfully extracted Walmart data')

    except FileNotFoundError as e:
        logging.error(f'File not found: {e}')
    
    except Exception as e:
        logging.error(f'Error during Walmart file extraction: {e}') 
    

In [4]:
walmart_sales = extraction()

In [8]:
def transformation(df):
        """
    Transforms raw Walmart sales data into a star schema format and prepares dimension and fact tables.

    Steps performed:
        - Extracts and formats date and time components.
        - Creates surrogate keys for dimension tables.
        - Constructs dimension tables: Store, Category, Payment, and Date.
        - Creates a fact table linking all dimension tables.
        - Adds a unified datetime column for analytics and machine learning purposes.

    Args:
        df (pd.DataFrame): Raw extracted Walmart sales data.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]: 
            - fact_walmart_sales: Fact table with keys and measurable data.
            - DimStore: Dimension table for store information.
            - DimCategory: Dimension table for product categories.
            - DimPayment: Dimension table for payment methods.
            - DimDate: Dimension table for date and time details.

    Raises:
        KeyError: If expected columns are missing in the input DataFrame.
        ValueError: If data conversion or datetime formatting fails.
    """
    
    logging.info('Start data cleaning process')
    
    try:
        df.dropna(inplace = True)
        df.drop_duplicates(inplace = True)
        logging.info('Dropped missing values and duplicates in data')
    
    except Exception as e:
        logging.error(f'Error during cleaning data: {e}')
    
    try:
        df['quantity'] = df['quantity'].astype(int)
        df['unit_price'] = df['unit_price'].replace('[\$,]' , '' , regex = True).astype(float)
        df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')
        df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')
        
        logging.info('Convert quantity, unit_price , date and time column into correct type')
    
    except ValueError as e:
        logging.error("ValueError during numeric conversion: %s", e)
     
    try:
        df['total_price'] = df['unit_price'] * df['quantity']
        df['revenue_per_item'] = df['total_price'] / df['quantity']
        df['profit'] = round(df['total_price'] * df['profit_margin'] , 2)
        
    
        df['DateTime'] = df['date'].dt.strftime('%Y-%m-%d') + ' ' + df['time'].dt.strftime('%H:%M:%S')
        df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y-%m-%d %H:%M:%S')
    
        df['day'] = df['DateTime'].dt.day
        df['day_of_week'] = df['DateTime'].dt.day_name()
        df['month'] = df['DateTime'].dt.month
        df['year'] = df['DateTime'].dt.year
        logging.info('Create new features correctly')
    
    except KeyError as e:
        logging.error("KeyError: Column not found - %s", e)
        
    # Create dimension tables
    DimStore = df[['Branch', 'City']].drop_duplicates().reset_index(drop=True)
    DimStore['store_id'] = DimStore.index + 1

    DimCategory = df[['category']].drop_duplicates().reset_index(drop=True)
    DimCategory['category_id'] = DimCategory.index + 10001

    DimPayment = df[['payment_method']].drop_duplicates().reset_index(drop=True)
    DimPayment['payment_id'] = DimPayment.index + 20001

    DimDate = df[['DateTime', 'year', 'month', 'day', 'day_of_week']].drop_duplicates().reset_index(drop=True)
    DimDate['date_id'] = DimDate.index + 30001

    # Create fact table
    fact_sales = df.copy()
    fact_sales = fact_sales.merge(DimStore, on=['Branch', 'City'], how='left')
    fact_sales = fact_sales.merge(DimCategory, on='category', how='left')
    fact_sales = fact_sales.merge(DimPayment, on='payment_method', how='left')
    fact_sales = fact_sales.merge(DimDate, on=['DateTime', 'year' , 'month', 'day', 'day_of_week'], how='left')

    fact_walmart_sales = fact_sales[['invoice_id','store_id', 'category_id', 'payment_id', 'date_id',
                                'rating', 'unit_price', 'quantity', 'profit_margin',
                                'total_price', 'revenue_per_item', 'profit']]

    logging.info('Transformation complete')
    return fact_walmart_sales, DimStore, DimCategory, DimPayment, DimDate


In [9]:
def load_data(fact_sales, DimStore, DimCategory, DimPayment, DimDate):
    """
    Loads the dimension and fact tables into an SQLite database and exports data marts as CSV files.

    Steps performed:
        - Saves all tables into an SQLite database (`walmart.db`).
        - Creates and exports a Machine Learning data mart (date vs profit).
        - Creates and exports an Analytics data mart with detailed transactional data.

    Args:
        fact_sales (pd.DataFrame): Fact table with sales metrics and foreign keys.
        DimStore (pd.DataFrame): Dimension table for store details.
        DimCategory (pd.DataFrame): Dimension table for category data.
        DimPayment (pd.DataFrame): Dimension table for payment methods.
        DimDate (pd.DataFrame): Dimension table with date-time breakdown.

    Returns:
        None

    Side Effects:
        - Writes to SQLite database (`walmart.db`).
        - Saves CSV files: `ml_data_mart.csv`, `analytics_data_mart.csv`.

    Raises:
        sqlite3.DatabaseError: If a database error occurs during table creation or querying.
        Exception: For general errors during the load process.
    """

    try:
        conn = sqlite3.connect('walmart.db')

        # Save dimension and fact tables
        DimStore.to_sql('DimStore', conn, index=False, if_exists='replace')
        DimCategory.to_sql('DimCategory', conn, index=False, if_exists='replace')
        DimPayment.to_sql('DimPayment', conn, index=False, if_exists='replace')
        DimDate.to_sql('DimDate', conn, index=False, if_exists='replace')
        fact_sales.to_sql('fact_walmart_sales', conn, index=False, if_exists='replace')
        
        logging.info("All tables saved to SQLite successfully")
        
        except Exception as e:
            logging.error(f"Error during loading data to SQLite: {e}")
        
        try:
            
            ml_query = """
            SELECT D.DateTime , F.profit
            FROM fact_walmart_sales F INNER JOIN DimDate D on F.date_id = D.date_id
            """
        
            ml_data_mart = pd.read_sql(ml_query , conn)
            ml_data_mart.to_csv('ml_data_mart.csv')
        
            analytics_query = """
            SELECT F.invoice_id, F.quantity, F.total_price, F.profit, C.category, F.rating, P.payment_method, D.Datetime  
            FROM fact_walmart_sales F INNER JOIN DimCategory C on F.category_id = C.category_id
            INNER JOIN DimPayment P ON F.payment_id = P.payment_id
            INNER JOIN DimDate D ON F.date_id = D.date_id
            """
            df_analytics = pd.read_sql(analytics_query , conn)
            df_analytics.to_csv('analytics_data_mart.csv')
        
            conn.commit()
            conn.close()
            logging.info("Data mart saved successfully")

        except Exception as e:
            logging.error(f"Error during loading data mart: {e}")
        
fact_sales, DimStore, DimCategory, DimPayment, DimDate = transformation(walmart_sales)
load_data(fact_sales, DimStore, DimCategory, DimPayment, DimDate)